In [1]:
import pandas as pd
import numpy as np
import pyodbc as pyo
import os
import geopandas as gpd
import shapely
import fiona
from pivottablejs import pivot_ui
from IPython.display import display
import openpyxl
import matplotlib.pyplot as plt

# import plotly.express as px
from shapely.geometry import Point, LineString, Polygon
pd.options.mode.chained_assignment = None  # default='warn'

### Reading the input data

In [2]:
data_path = os.path.abspath(os.path.join( os.getcwd() , '..', 'Data'))
db_file = os.path.join(data_path,'2017-20 pooled SEQ QTS.accdb')
spatial_zone_files = os.path.abspath( r"C:\\Users\\taghavm\\OneDrive - Jacobs\\Documents\\Projects\\SCHTS\\WORKING\\Data/inputs/spatial/2250trafficzonesandcentriods_zone/2250trafficzonesandcentriods_zone.SHP")


In [3]:
%%time
cnxn = pyo.connect('DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={};'.format(db_file))
cursor = cnxn.cursor();tabs = cursor.tables();tablelist = []
for i in tabs:
    if i.table_name.startswith("MSys"):
        pass
    else:
#         print(i.table_name)
        tablelist.append(i.table_name)
del i, tablelist, tabs

Wall time: 104 ms


In [4]:
%%time
"""Import HTS Data."""

HOUSEHOLDS = pd.read_sql('SELECT * from {}'.format('1_QTS_HOUSEHOLDS'), con = cnxn).fillna(0)
PERSONS = pd.read_sql('SELECT * from {}'.format('2_QTS_PERSONS'), con = cnxn).fillna(0)
#VEHICLES = pd.read_sql('SELECT * from {}'.format('3_QTS_VEHICLES'), con = cnxn).fillna(0)
STOPS = pd.read_sql('SELECT * from {}'.format('4_QTS_STOPS'), con = cnxn).fillna(0)
TRIPS = pd.read_sql('SELECT * from {}'.format('5_QTS_TRIPS'), con = cnxn).fillna(0)
ANZSCO = pd.read_sql('SELECT * from {}'.format('R_ANZSCO'), con = cnxn).fillna(0)
ASGS = pd.read_sql('SELECT * from {}'.format('R_ASGS_2016'), con = cnxn).fillna(0)
#DIST_bin = pd.read_sql('SELECT * from {}'.format('R_Distance_bins'), con = cnxn).fillna(0)
#MAINACT = pd.read_sql('SELECT * from {}'.format('R_GROUP_MAINACT'), con = cnxn).fillna(0)
#GRP_MODE = pd.read_sql('SELECT * from {}'.format('R_GROUP_MODE'), con = cnxn).fillna(0)
#LGA = pd.read_sql('SELECT * from {}'.format('R_LGA'), con = cnxn).fillna(0)
#MAINMODE = pd.read_sql('SELECT * from {}'.format('R_MAINMODE'), con = cnxn).fillna(0)
#PURPOSE = pd.read_sql('SELECT * from {}'.format('R_OVERALL_PURPOSE'), con = cnxn).fillna(0)
#REGION = pd.read_sql('SELECT * from {}'.format('R_REGION'), con = cnxn).fillna(0)

TIME = pd.read_excel('R_TIME.xlsx', engine = 'openpyxl').fillna(0)

Wall time: 8.46 s


In [5]:
# """Adjust PERSID."""
# PERSONS['PERSID'] = PERSONS['PERSID'].str.replace('/','')
# TRIPS['PERSID'] = TRIPS['PERSID'].str.replace('/','')
# STOPS['PERSID'] = STOPS['PERSID'].str.replace('/','')

In [6]:
# PERSONS.drop(columns = ['STARTPLACE', 'ANYSTOPS', 'REASON', 'LASTLEFT','RELATIONSHIP', #'CARLICENCE', 'CARLICTYPE','MCLICENCE', 'MCLICTYPE', 'OTHERLICENCE',
#                         'YEAROFBIRTH','ASSISTAGE', 'ASSISTLTHC', 'ASSISTSTHC',
#                         'ASSISTDISABILITY','ASSISTENGLISH', 'ASSISTOTHER', 'ASSISTANY','RIDESHARETRIPS', 'RIDESHAREENT', 
#                         'RIDESHAREHC', 'RIDESHAREED', 'RIDESHARESHOP', 'RIDESHAREWORK', 'RIDESHAREOTHER', 'TAXITRIPSS',
#                         'TAXIENT', 'TAXIHC', 'TAXIED', 'TAXISHOP', 'TAXIWORK', 'TAXIOTHER'], 
#              inplace = True)

# HOUSEHOLDS.drop(columns= ['BIKES', 'HHVEH', 'DWELLTYPE', 'SURVEYWEEK',
#        'STRATA_LGA', 'TRAVDATE', 'TRAVMONTH', 'TRAVYEAR', 'TRAVDOW'], inplace = True)

In [7]:
"""Load New Weightings"""
HH = pd.read_excel('Reweight.xlsx', engine='openpyxl', sheet_name='HH_export')
Pers = pd.read_excel('Reweight.xlsx', engine='openpyxl', sheet_name='Pers_export')
wgt_sect = pd.read_excel('Reweight.xlsx', engine='openpyxl', sheet_name='WGT_Sect')

In [8]:
"""Load Spatial Zones"""
zones = gpd.read_file(spatial_zone_files)
zones = zones.to_crs(epsg=4283) # to be able to do spatial matching with the x,y coordinates 


# Data Table Cleaning

## Households Table 

- Joining the household table with the zone system.

- And Updating the weighting factors in the HTS table to reflect the correct weights

In [9]:
Household = HOUSEHOLDS.copy()
Household = Household.astype({'HHID':'int64'})
Household = pd.merge(Household, ASGS, left_on = 'HOME_SA1_2016', right_on = 'SA1_7DIGITCODE_2016')

"""Spatial Join to Model Sectors"""
Household_map = gpd.GeoDataFrame(Household, geometry=gpd.points_from_xy(Household['HOMELONG'], Household['HOMELAT']), crs='EPSG:4283')
Household_map = gpd.sjoin(Household_map, zones[['geometry','NO', 'CODE', 'NAME', 'TYPENO', 'SCHOOLNAME','SCHOOLTYPE']],
 how = 'left', op = 'within').drop(columns ='index_right')

In [10]:
Household_map.columns

Index(['HHID', 'HHSIZE', 'BIKES', 'HHVEH', 'DWELLTYPE', 'SURVEYWEEK',
       'STRATA_LGA', 'TRAVDATE', 'TRAVMONTH', 'TRAVYEAR', 'TRAVDOW',
       'HOMELONG', 'HOMELAT', 'HOME_SA1_2016', 'HHWGT_19',
       'SA1_7DIGITCODE_2016', 'SA2_5DIGITCODE_2016', 'SA2_NAME_2016',
       'SA3_CODE_2016', 'SA3_NAME_2016', 'SA4_CODE_2016', 'SA4_NAME_2016',
       'LGA_NAME_2016', 'geometry', 'NO', 'CODE', 'NAME', 'TYPENO',
       'SCHOOLNAME', 'SCHOOLTYPE'],
      dtype='object')

In [11]:
Household = pd.DataFrame(Household_map).drop(columns = 'geometry')
Household.rename(columns = {'NO':'Home_Zone_number',  'SA2_NAME_2016':'HH_SA2', 'SA3_NAME_2016': 'HH_SA3', 'SA4_NAME_2016': "HH_SA4", 'LGA_NAME_2016': "HH_LGA"}, inplace = True)

Household.drop(columns=[ 'SA2_5DIGITCODE_2016', 'SA3_CODE_2016', 'SA4_CODE_2016'], inplace=True)

In [12]:
print('Number of Household Records in Original: {}'.format(len(HOUSEHOLDS.axes[0])))
print('Number of Household Records: {}'.format(len(Household.axes[0])))

Number of Household Records in Original: 14715
Number of Household Records: 14715


In [13]:
"""Update Household Weights"""
Household['old_HHWGT_19'] = Household['HHWGT_19']
Household = Household.set_index("HHID")
Household.update(HH.set_index('HHID'))
Household.reset_index(inplace=True)

print("LGA Weighting Population: {}".format(Household.old_HHWGT_19.sum().round(0)))
print("Sector Weighting Population: {}".format(Household.HHWGT_19.sum().round(0)))

del Household_map
Household[['HH_LGA','old_HHWGT_19','HHWGT_19']].groupby(by='HH_LGA').agg(['count', 'sum']).round(0)

LGA Weighting Population: 1167298.0
Sector Weighting Population: 1166010.0


old_HHWGT_19           HHWGT_19          
                          count       sum    count       sum
HH_LGA                                                      
Brisbane (C)               2963  427770.0     2963  427403.0
Gold Coast (C)             2038  214279.0     2038  218276.0
Ipswich (C)                1747   70732.0     1747   69231.0
Logan (C)                  1702  103359.0     1702  102988.0
Moreton Bay (R)            1896  157796.0     1896  157320.0
Noosa (S)                   956   20877.0      956   23174.0
Redland (C)                1655   54725.0     1655   54598.0
Sunshine Coast (R)         1758  117760.0     1758  113020.0

## Person Table

- Adding ANZCO table to indentify the Blue and White Collar workers
- Addining the new person weight 
- Persons Table updated by Childcare and Kindergarden-age children 
- Update Work and Study status

In [14]:
""" Blue/White Collar Adjustment."""
adjustpath = os.path.join(data_path, "inputs")
try:
    ANZSCO_filter = pd.read_csv(os.path.join(adjustpath, 'ANZSCO_adjustment.csv'))
    print('Using existing Blue/White Collar split table')
    #os.startfile(os.path.join(adjustpath, 'ANZSCO_adjustment.csv'))
except IOError:
    print('Rerun after completing Blue/White Collar split table')
    ANZSCO.reindex(ANZSCO.columns.tolist() + ['Collar', 'Collar2'], axis=1).drop(columns=['ANZSCO_2','Description_2',
    'ANZSCO_4','Description_4','ANZSCO_6', 'Description_6']).drop_duplicates().to_csv(os.path.join(adjustpath,'ANZSCO_adjustment.csv'), index= False)
    os.startfile(os.path.join(adjustpath, 'ANZSCO_adjustment.csv'))


Using existing Blue/White Collar split table


In [15]:
print(os.path.join(adjustpath, 'ANZSCO_adjustment.csv'))

C:\Users\taghavm\OneDrive - Jacobs\Documents\Projects\SCHTS\WORKING\Data\inputs\ANZSCO_adjustment.csv


In [16]:
ANZSCO_filter.loc[~pd.isna(ANZSCO_filter.Collar)]

,ANZSCO_1,Description_1,ANZSCO_3,Description_3,Collar,Collar2
0,1,MANAGERS,100,Managers nfd,White,Professional
1,1,MANAGERS,141,Accommodation and Hospitality Managers,White,White
2,1,MANAGERS,149,"Miscellaneous Hospitality, Retail and Service ...",White,White
3,1,MANAGERS,132,Business Administration Managers,White,Professional
4,1,MANAGERS,131,"Advertising, Public Relations and Sales Managers",White,Professional
...,...,...,...,...,...,...
134,8,LABOURERS,891,Freight Handlers and Shelf Fillers,Blue,Blue
135,8,LABOURERS,890,Other Labourers nfd,Blue,Blue
136,9,MISCELLANEOUS,999,Response Unidentifiable,White,White
137,9,MISCELLANEOUS,997,Response Unidentifiable,White,White


In [17]:
Person = PERSONS.copy()
Person = Person.astype({'HHID':'int64', "PERSID": 'str', 'ANZSCO_1-digit': int, 'ANZSCO_3-digit': 'int64'}).sort_values('HHID')

In [18]:
Household.columns

Index(['HHID', 'HHSIZE', 'BIKES', 'HHVEH', 'DWELLTYPE', 'SURVEYWEEK',
       'STRATA_LGA', 'TRAVDATE', 'TRAVMONTH', 'TRAVYEAR', 'TRAVDOW',
       'HOMELONG', 'HOMELAT', 'HOME_SA1_2016', 'HHWGT_19',
       'SA1_7DIGITCODE_2016', 'HH_SA2', 'HH_SA3', 'HH_SA4', 'HH_LGA',
       'Home_Zone_number', 'CODE', 'NAME', 'TYPENO', 'SCHOOLNAME',
       'SCHOOLTYPE', 'old_HHWGT_19'],
      dtype='object')

In [19]:
Person = pd.merge(Person, Household[['HHID', 
                                     'HH_SA2', 
                                     'HH_SA3', 
                                     'HH_SA4', 
                                     'HH_LGA',
                                     ]], left_on = 'HHID', right_on = 'HHID', how = 'left')

"""Fix Random Errors Here."""
Person.loc[Person.PERSID == 613521001,'ANZSCO_1-digit'] = 2
Person.loc[Person.PERSID == 613521001,'ANZSCO_3-digit'] = 200

In [20]:
Person.replace(['primary','secondary'],['Primary','Secondary'], inplace=True)
"""Fix Random Errors Here."""

"""Add Collar Tag."""
%time Person = pd.merge(Person, ANZSCO_filter[['ANZSCO_3','Collar', 'Collar2']], left_on = 'ANZSCO_3-digit', right_on = 'ANZSCO_3', how = 'left').drop(columns= 'ANZSCO_3').sort_values('HHID').drop_duplicates().dropna(axis = 0, thresh = 5)

Person = Person.reset_index().drop(columns = 'index')

Wall time: 406 ms


In [21]:
"""Update New Person Weights."""
Person['old_PERSWGT19'] = Person['PERSWGT19']
Person = Person.set_index("PERSID")
Person.update(Pers.set_index('PERSID'))
Person.reset_index(inplace=True)

print('Number of Person Records in Original: {}'.format(len(PERSONS.axes[0])))
print('Number of Person Records: {}'.format(len(Person.axes[0])))

Number of Person Records in Original: 36264
Number of Person Records: 36264


___TODO :  Check if all the workers have a blue/white collar code.___

Persons Table Allocate Childcare and Kindergarden-age children

In [22]:
"""Calculate Kindergarten/Childcare kids."""
trip_rev = TRIPS.copy()
kindy_pers = Person.copy()

"""Age Bands."""
under5 = kindy_pers.AGE < 5

"""Processing."""
duration = trip_rev.DURATION > 90
DEST_CC_kindy = trip_rev.DESTPLACE == 'Childcare or kindergarten'

CC_person = trip_rev[['PERSID',"HHID", "DESTPLACE", "DURATION"]].loc[duration & DEST_CC_kindy,['PERSID']].drop_duplicates().astype({'PERSID': 'str'})
CC_person['CC/Kindy'] = 1

kindy_pers = pd.merge(kindy_pers, CC_person, on='PERSID', how='left')
CC_person = kindy_pers["CC/Kindy"] == 1
not_CC_kid = kindy_pers.STUDYING != 'CC/Kindy'

kindy_pers.loc[CC_person & under5 & not_CC_kid,'STUDYING'] = 'CC/Kindy'
kindy_pers.loc[CC_person & under5 & not_CC_kid,'Student_Prim'] = 0
kindy_pers.loc[CC_person & under5 & not_CC_kid,'Student_CC'] = 1
print("Number of Added Kindy Kids:{}".format(len(kindy_pers.loc[CC_person & under5 & not_CC_kid])))
kindy_pers.drop(columns=['CC/Kindy'], inplace = True)

#Assign Kindy
kindy = kindy_pers.STUDYING == 'CC/Kindy'
kindy_pers.loc[kindy, 'Student_CC'] = 1

Person = kindy_pers
del trip_rev, kindy_pers, kindy, not_CC_kid, under5, duration, DEST_CC_kindy, CC_person


Number of Added Kindy Kids:368


In [23]:
test = Person.loc[Person.Student_CC==1]
test.HH_SA4.value_counts()

Gold Coast             64
Sunshine Coast         54
Ipswich                47
Logan - Beaudesert     44
Brisbane - East        33
Brisbane - West        26
Brisbane - South       25
Brisbane Inner City    23
Moreton Bay - South    22
Brisbane - North       17
Moreton Bay - North    13
Name: HH_SA4, dtype: int64

In [24]:
"""Calculate Primary kids."""
trip_rev = TRIPS.copy()
school_pers = Person.copy()
school_pers['STUDYING'].replace({'other': 0}, inplace = True)

"""Age Bands."""
under6 = school_pers.AGE < 5
over5 = school_pers.AGE >= 5
under13 = school_pers.AGE < 13
"""Processing."""
duration = trip_rev.DURATION > 90
zero_study = school_pers.STUDYING == 0
DEST_prim= trip_rev.DESTPLACE == 'Primary or secondary school'

prim_person = trip_rev.loc[DEST_prim & duration,['PERSID']].drop_duplicates().astype({'PERSID': 'str'})
prim_person['Prim_person'] = 1
school_pers = pd.merge(school_pers, prim_person, on='PERSID', how='left')
missing = school_pers.STUDYING == 0

prim_missing = school_pers.Prim_person == 1

school_pers.loc[missing & prim_missing & over5 & under13, 'STUDYING'] = 'Primary'
school_pers.loc[missing & prim_missing & under6, 'STUDYING'] = 'CC/Kindy'
print("Number of Added Kindy Kids:{}".format(len(school_pers.loc[missing & prim_missing & under6])))
print("Number of Added Primary Kids:{}".format(len(school_pers.loc[missing & prim_missing & over5 & under13])))

school_pers.drop(columns = 'Prim_person', inplace = True)

Person = school_pers

del school_pers, prim_person, prim_missing, DEST_prim, under6, over5, under13, trip_rev, missing

Number of Added Kindy Kids:52
Number of Added Primary Kids:24


In [27]:
TRIPS.columns

Index(['TRIPID', 'HHID', 'PERSID', 'STARTSTOP', 'ENDSTOP', 'STARTIME',
       'ORIGPLACE', 'ORIGPURP', 'ORIGLONG', 'ORIGLAT', 'ORIGSA1_2016',
       'DESTPLACE', 'DESTPURP', 'DESTLONG', 'DESTLAT', 'DESTSA1_2016',
       'MAINMODE', 'MODE1', 'MODE2', 'MODE3', 'MODE4', 'MODE5', 'MODE6',
       'MODE7', 'MODE8', 'ARRTIME', 'DURATION', 'TRAVTIME', 'CUMDIST',
       'OVERALL_PURPOSE', 'TRIPWGT19'],
      dtype='object')

In [ ]:
pers_filt = Person.copy()

"""Age Bands."""
over5 = pers_filt.AGE > 4
under5 = pers_filt.AGE < 5
under18 = pers_filt.AGE < 18
over15 = pers_filt.AGE > 14
sub75 = pers_filt.AGE < 75
over75 = pers_filt.AGE > 74
under15 = pers_filt.AGE < 15
over20 = pers_filt.AGE > 20
o13 = pers_filt.AGE > 12
u13 = pers_filt.AGE < 13

"""Clean Studying Var."""
PrimSchl = pers_filt.STUDYING == 'Primary'
SecSchl = pers_filt.STUDYING == 'Secondary'
UniFT = pers_filt.STUDYING == 'tertiaryFullTime'
UniPT = pers_filt.STUDYING == 'tertiaryPartTime'
CC_kindy = pers_filt.MAINACT == 'Childcare/Kindergarten'

# print('CC/Kindy')
# print(pers_filt.loc[CC_kindy,['STUDYING','AGE']].value_counts())
pers_filt.loc[CC_kindy,'STUDYING'] = "CC/Kindy"
# print("Primary to Kindy")
# print(pers_filt.loc[PrimSchl & under5, ['STUDYING','AGE']].value_counts())
pers_filt.loc[(PrimSchl | SecSchl | UniFT | UniPT) & under5,'STUDYING'] = "CC/Kindy"
# print("Primary to Secondary")
# print(pers_filt.loc[PrimSchl & o13, ['STUDYING','AGE']].value_counts())
pers_filt.loc[PrimSchl & o13,'STUDYING'] = 'Secondary'
# print("Adult Secondary")
# print(pers_filt.loc[SecSchl & over20, ['STUDYING','AGE']].value_counts())
pers_filt.loc[SecSchl & over20,'STUDYING'] = 0
# print("Secondary to Primary")
# print(pers_filt.loc[SecSchl & u13, ['STUDYING','AGE']].value_counts())
pers_filt.loc[(SecSchl| UniFT | UniPT) & u13,'STUDYING'] = 'Primary'
# print('Uni errors')
# print(pers_filt.loc[UniFT & u13, ['STUDYING','AGE']].value_counts())
pers_filt.loc[UniFT & u13,'STUDYING'] = 0
# print(pers_filt.loc[UniPT & u13, ['STUDYING','AGE']].value_counts())
pers_filt.loc[UniPT & u13,'STUDYING'] = 0

#fig = px.histogram(pers_filt[['STUDYING',"MAINACT",'AGE']], x="AGE", color="STUDYING", marginal = "rug", hover_data=['MAINACT']);fig.show()
"""Assign Study Vars."""
PrimSchl = pers_filt.STUDYING == 'Primary'
SecSchl = pers_filt.STUDYING == 'Secondary'
TertPT = pers_filt.STUDYING == 'tertiaryFullTime'
TertFT = pers_filt.STUDYING == 'tertiaryPartTime'

pers_filt.loc[PrimSchl, 'Student_Prim'] = 1
pers_filt.loc[SecSchl, 'Student_Sec'] = 1
pers_filt.loc[TertPT | TertFT, 'Student_Tert'] = 1
pers_filt.loc[PrimSchl | SecSchl | TertPT | TertFT, 'Student'] = 1

"""Clean Worker Variable."""
retired = pers_filt.WORKSTATUS == 'retired'
PrimSchl = pers_filt.STUDYING == 'Primary'
Not_PrimSchl = pers_filt.STUDYING != 'Primary'

#display(pers_filt.loc[PrimSchl| retired | under15, ['Collar','AGE']].plot.hist(bins=20))
# pers_filt.loc[PrimSchl| retired | under15,'Collar'] = np.NaN
pers_filt.loc[retired,'Collar'] = np.NaN

white = pers_filt.Collar == 'White'
blue = pers_filt.Collar == 'Blue'

"""Assign Worker vs Dependents."""
workFT = pers_filt.WORKSTATUS == 'workFullTime'
workPT = pers_filt.WORKSTATUS == 'workPartTime'
non_workFT = pers_filt.WORKSTATUS != 'workFullTime'
non_workPT = pers_filt.WORKSTATUS != 'workPartTime'

pers_filt.loc[(non_workFT & non_workPT) | PrimSchl | under15, 'Dependent'] = 1
pers_filt.loc[(workFT | workPT) & Not_PrimSchl & over15, 'Worker'] = 1
worker = pers_filt.Worker == 1

pers_filt.loc[worker & white, 'Worker_White'] = 1
pers_filt.loc[worker & blue, 'Worker_Blue'] = 1

"""Apply Age Categories."""
u15 = pers_filt.AGE < 15
o15 = pers_filt.AGE > 14

u30 = pers_filt.AGE < 30
o30 = pers_filt.AGE > 29

u50 = pers_filt.AGE < 50
o50 = pers_filt.AGE > 49

u75 = pers_filt.AGE < 75
o75 = pers_filt.AGE > 74

pers_filt.loc[u15, '0-14'] = 1
pers_filt.loc[o15 & u30, '15-29'] = 1
pers_filt.loc[o30 & u50, '30-49'] = 1
pers_filt.loc[o50 & u75, '50-74'] = 1
pers_filt.loc[o75, '75+'] = 1

pers_filt.loc[u15,'Age_cat'] = '0-14'
pers_filt.loc[o15 & u30,'Age_cat'] =  '15-29'
pers_filt.loc[o30 & u50,'Age_cat'] =  '30-49'
pers_filt.loc[o50 & u75,'Age_cat'] =  '50-74'
pers_filt.loc[o75,'Age_cat'] =  '75+'


#Missing Record Check
pers_filt.fillna(0, inplace = True)
pers_filt['check'] = pers_filt['Worker'] + pers_filt['Dependent']
print('ERRORS PRESENT:{} Person Records without Variable attached'.format(pers_filt[pers_filt.check!=1].PERSID.count()))
print('REVIEW PERSID:{} '.format(pers_filt[pers_filt.check!=1].PERSID.to_list()))
pers_filt.drop(columns = ['check'], inplace = True)

Person = pers_filt
del PrimSchl, SecSchl, TertPT, TertFT
del workFT, workPT, non_workFT, non_workPT, white, blue
del u15, o15,u30, o30, u50, o50, u75, o75
del pers_filt
del over5, under5, under18, over15, sub75, over75, under15, over20, o13, u13
del retired, worker, UniFT, UniPT

In [ ]:
"""Calculate Workers."""
trip_rev = TRIPS.copy()
work_pers = Person.copy()

"""Age Bands."""
over15 = work_pers.AGE > 14

"""Processing."""
duration = trip_rev.DURATION > 60 #--------was 60-----------------------------------------------------------------------------------
DEST_WP = trip_rev.DESTPLACE == 'My workplace'
Not_PrimSchl = work_pers.STUDYING != 'Primary'

work_person = trip_rev[['PERSID',"HHID", "DESTPLACE", "DURATION"]].loc[duration & DEST_WP,['PERSID']].drop_duplicates().astype({'PERSID': 'str'})

work_person['does_work'] = 1

work_pers = pd.merge(work_pers, work_person, on='PERSID', how='left')
work_person_type = work_pers['does_work'] == 1
not_worker = work_pers.Worker != 1
blue_col = work_pers.Collar == 'Blue'
white_col = work_pers.Collar == 'White'
no_col = work_pers.Collar == 0
work_pers.loc[work_person_type & over15 & Not_PrimSchl & not_worker,'Worker'] = 1
work_pers.loc[work_person_type & over15 & Not_PrimSchl & not_worker & blue_col,'Worker_Blue'] = 1
work_pers.loc[work_person_type & over15 & Not_PrimSchl & not_worker & white_col,'Worker_White'] = 1
work_pers.loc[work_person_type & over15 & Not_PrimSchl & not_worker & no_col,'Worker_White'] = 1
print("Number of Added Workers:{}".format(len(work_pers.loc[work_person_type & over15 & not_worker])))
print("Number of Added Workers - White:{}".format(len(work_pers.loc[work_person_type & over15 & not_worker & white_col])))
print("Number of Added Workers - Blue:{}".format(len(work_pers.loc[work_person_type & over15 & not_worker & blue_col])))
print("Number of Added Workers - None:{}".format(len(work_pers.loc[work_person_type & over15 & not_worker & no_col])))
work_pers.drop(columns=['does_work'], inplace = True)

Person = work_pers
del work_person, trip_rev, over15, blue_col, white_col, no_col, work_pers, work_person_type, not_worker, DEST_WP, duration

In [ ]:
grouped_person = Person[['PERSID','0-14', '15-29', '30-49','50-74', '75+','Age_cat', 'Dependent', 'Worker', 'Student','Worker_Blue', 'Worker_White', 'STUDYING' ,'Student_Prim', 'Student_Sec','Student_Tert','Student_CC', 'AGE','HHID']].groupby('HHID').agg({'PERSID': 'count','0-14': 'sum', '15-29': 'sum', '30-49': 'sum','50-74': 'sum', '75+': 'sum', 'Worker_Blue':'sum', 'Worker_White': 'sum','Worker': 'sum', 'Student_Prim': 'sum', 'Student_Sec': 'sum','Student_Tert': 'sum','Student_CC': 'sum','Student': 'sum', 'AGE': 'mean'}).rename(columns={'PERSID':'Residents'})
Household = pd.merge(Household, grouped_person, left_on = 'HHID', right_index = True)
del grouped_person


## Trips Table

1. Identify the HomeBased Trips
2. Additional Flags 
    2.1. Adding a work flag  
    2.2. Adding Activity Duration flags  
    2.3. Work flag  
    2.4. Escort flag  
    2.5. Others flag  
    
    
    
    


In [ ]:
trip = TRIPS.copy()
trip = trip.astype({'TRIPID': 'int64', 'HHID':'int64', "PERSID": 'str'})
trip['trip_stops'] = trip['ENDSTOP'] - trip['STARTSTOP']
trip = pd.merge(trip, Person[['PERSID','Collar','Collar2','WORKSTATUS', 'STUDYING',
                              '0-14', '15-29', '30-49','50-74', '75+','Age_cat', 
                              'Dependent', 'Worker', 'Student','Worker_Blue', 'Worker_White', 
                              'Student_Prim', 'Student_Sec','Student_Tert', 'Student_CC',
                              'AGE','HHID', 'HH_SA2', 'HH_SA3', 'HH_SA4', 'HH_LGA']], left_on = 'PERSID', right_on = 'PERSID', how='left').drop(columns='HHID_y').rename(columns={'HHID_x': 'HHID','SA2_NAME_2016': "HH_SA2", 'SA3_NAME_2016': "HH_SA3", 'SA4_NAME_2016': "HH_SA4", 'LGA_NAME_2016': "HH_LGA"})

print('Number of Trip Records in Original: {}'.format(len(TRIPS.axes[0])))
print('Number of Trip Records: {}'.format(len(trip.axes[0])))

trip['old_TRIPWGT19'] = trip['TRIPWGT19']
trip = trip.set_index("PERSID")
TripPers = Pers.rename(columns={'PERSWGT19':'TRIPWGT19'})

In [ ]:
trip.update(TripPers.set_index('PERSID'))
trip.reset_index(inplace=True)
del TripPers

In [ ]:
"""Purpose Adjustments."""
purp_filt = trip.copy()
dur_u15 = purp_filt.DURATION < 16
dur_o15 = purp_filt.DURATION > 15
dur_o60 = purp_filt.DURATION > 59
dur_o180 = purp_filt.DURATION > 179

purp_filt.loc[dur_u15, 'Dur'] = 'u15'
purp_filt.loc[dur_o15, 'Dur'] = 'o15'
purp_filt.loc[dur_o60, 'Dur'] = 'o60'
purp_filt.loc[dur_o180, 'Dur'] = 'o180'
purp_filt.head()

### Home Based Definition
As it is defined below, home based flag represents the records which either their **origin place** or **destination place** is "My home", in addition to the records which either their **origin purpose** or **destination purpose** is "At home" or "Go home". 

In [ ]:
""" Home Based
    Logic:
    Place: 'My Home' = Home Based trip
    ORIGPLACE: 'My Home' -> P2A Trip -> HB_Orig Flag
    DESTPLACE: 'My Home' -> A2P Trip -> HB_Dest Flag
    Flags:
    HB_Orig or HB_Dest = 'Home_based' Flag
"""
home_is_orig = purp_filt.ORIGPLACE == 'My home'
purp_filt.loc[home_is_orig, 'HB_Orig'] = 1
home_is_dest = purp_filt.DESTPLACE == 'My home'
purp_filt.loc[home_is_dest, 'HB_Dest'] = 1
purp_filt.loc[home_is_orig | home_is_dest, 'Home_based'] = 1


purp_filt.loc[home_is_orig, 'Orig_Type'] = 'Home'
purp_filt.loc[home_is_dest, 'Dest_Type'] = 'Home'

home_purp_is_orig = purp_filt.ORIGPURP == 'At Home'
purp_filt.loc[home_purp_is_orig, 'HB_Orig'] = 1
home_purp_is_dest = purp_filt.DESTPURP == 'Go home'
purp_filt.loc[home_purp_is_dest, 'HB_Dest'] = 1
purp_filt.loc[home_purp_is_orig | home_purp_is_dest, 'Home_based'] = 1

del home_is_orig, home_is_dest, home_purp_is_orig, home_purp_is_dest

### Work Flag Definition
As it is defined below, work flag represents the records which either their **origin purpose** or **destination purpose** is "Work (my workplace)" or "Work (other work reason)". 

In [ ]:
""" Work Trip
    Logic:
    PLACE: 'My Workplace' = Work Trip
    PLACE: 'Other work-related place' = Work Trip
    Flags:
    ORIG/DEST Workplace or other Related = 'work' Flag
"""
work_orig = purp_filt.ORIGPLACE == 'My workplace' 
work_dest = purp_filt.DESTPLACE == 'My workplace'
work_orig_other = purp_filt.ORIGPLACE == 'Other work-related place'
work_dest_other = purp_filt.DESTPLACE == 'Other work-related place'

# purp_filt.loc[work_orig | work_dest | work_orig_other | work_dest_other, 'work'] = 1
# purp_filt.loc[work_orig | work_orig_other, 'Orig_Type'] = 'work'
# purp_filt.loc[work_dest | work_dest_other, 'Dest_Type'] = 'work'

## last three lines are being commented since the work flag should be True if the purpose is "work" not 
## the places. In some record, even though the origplace or destplace is work-related, the purpose for the trip is not working.

del work_orig, work_dest, work_dest_other, work_orig_other


#Work Purpose
work_o_purp = purp_filt.ORIGPURP == 'Work (my workplace)'
work_d_purp = purp_filt.DESTPURP == 'Work (my workplace)'
work_o_purp2 = purp_filt.ORIGPURP == 'Work (other work reason)'
work_d_purp2 = purp_filt.DESTPURP == 'Work (other work reason)'

## Modification: The nest three lines are added
purp_filt.loc[work_o_purp | work_d_purp | work_o_purp2 | work_d_purp2, 'work'] = 1
purp_filt.loc[work_o_purp | work_o_purp2, 'Orig_Type'] = 'work'
purp_filt.loc[work_d_purp | work_d_purp2, 'Dest_Type'] = 'work'

del work_o_purp, work_d_purp, work_o_purp2, work_d_purp2
purp_filt.head()

### Education Definitions
* Education Purpose flag represents the records which either their **origin purpose** or **destination purpose** is "Education". 

* **Edu_cps_place** flags represent the records either their origin or destination is whitin the group of places including: Primary or secondary school, Childcare or kindergarten and Other study-related place. 

**Note**:

 **Edu_p** flag represents the primary students where one of the following places are the origin or the destination of a trip:
    * Primary or secondary school
    * Other study-related place
    * My campus
    
 **Edu_s** flag represents the secondary students where one of the following places are the origin or the destination of a trip:
    * Primary or secondary school
    * Other study-related place
    * My campus

**These flags are used in defining HBEp and HBEs category. 
There are 3 trips through which a primary student with the purpose of Education goes to or comes from My campus.
There are 22 trips through which a secondary student with the purpose of Education goes to or comes from My campus. 
Since the age of trip makers falls within the acceptable range we have kept these records untouched**

In [ ]:
##Education Flag
#Purpose
educ_purp_o = purp_filt.ORIGPURP == 'Education'
educ_purp_d = purp_filt.DESTPURP == 'Education'

purp_filt.loc[educ_purp_o | educ_purp_d, 'Edu_purp'] = 1

""" Primary/Secondary
    Logic: 
    Place: 'Primary or secondary school' = HBE Trip
    Place: 'Childcare or kindergarten' = HBE Trip
    Flags:
    ORIG/DEST Prim/Sec or Child/Kindy = "Edu_p/s" Flag
"""
prim = purp_filt.Student_Prim == 1
sec = purp_filt.Student_Sec == 1
tert = purp_filt.Student_Tert == 1
CC_kids = purp_filt.Student_CC == 1
non_student = purp_filt.Student != 1
purp_filt['Student_ALL'] = purp_filt['Student_Prim'] + purp_filt['Student_Sec'] + purp_filt['Student_Tert'] + 
purp_filt['Student_CC']
non_all_student = purp_filt.Student_ALL == 0
purp_filt['Student_Young'] = purp_filt['Student_Prim'] + purp_filt['Student_Sec'] + purp_filt['Student_CC']
non_young_student = purp_filt.Student_Young == 0

primsec_orig = purp_filt.ORIGPLACE == 'Primary or secondary school' 
primsec_dest = purp_filt.DESTPLACE == 'Primary or secondary school' 
CCKindy_orig = purp_filt.ORIGPLACE == 'Childcare or kindergarten' 
CCKindy_dest = purp_filt.DESTPLACE == 'Childcare or kindergarten'
other_ed_orig = purp_filt.ORIGPLACE == 'Other study-related place' 
other_ed_dest = purp_filt.DESTPLACE == 'Other study-related place'
campus_ed_orig = purp_filt.ORIGPLACE == 'My campus' 
campus_ed_dest = purp_filt.DESTPLACE == 'My campus'

# purp_filt.loc[primsec_orig | primsec_dest | CCKindy_orig | CCKindy_dest | other_ed_orig | other_ed_dest | campus_ed_orig | campus_ed_dest, 'Edu_place'] = 1

purp_filt.loc[primsec_orig | primsec_dest | CCKindy_orig | CCKindy_dest | other_ed_orig | other_ed_dest, 'Edu_cps_place'] = 1

purp_filt.loc[primsec_orig | CCKindy_orig | other_ed_orig | campus_ed_orig , 'Orig_Type'] = 'Education'
purp_filt.loc[primsec_dest | CCKindy_dest | other_ed_dest | campus_ed_dest, 'Dest_Type'] = 'Education'

"""
Categorise education trips by students
"""
# educ_O_D = purp_filt.Edu_place == 1
purp_filt.loc[primsec_orig & prim, 'Edu_p'] = 1
purp_filt.loc[primsec_dest & prim, 'Edu_p'] = 1
purp_filt.loc[other_ed_orig & prim, 'Edu_p'] = 1
purp_filt.loc[other_ed_dest & prim, 'Edu_p'] = 1
purp_filt.loc[campus_ed_orig & prim, 'Edu_p'] = 1
purp_filt.loc[campus_ed_dest & prim, 'Edu_p'] = 1

purp_filt.loc[primsec_orig & sec, 'Edu_s'] = 1
purp_filt.loc[primsec_dest & sec, 'Edu_s'] = 1
purp_filt.loc[other_ed_orig & sec, 'Edu_s'] = 1
purp_filt.loc[other_ed_dest & sec, 'Edu_s'] = 1
purp_filt.loc[campus_ed_orig & sec, 'Edu_s'] = 1
purp_filt.loc[campus_ed_dest & sec, 'Edu_s'] = 1

purp_filt.loc[other_ed_orig & tert, 'Edu_t'] = 1
purp_filt.loc[other_ed_dest & tert, 'Edu_t'] = 1
purp_filt.loc[campus_ed_orig & tert, 'Edu_t'] = 1
purp_filt.loc[campus_ed_dest & tert, 'Edu_t'] = 1

purp_filt.loc[CCKindy_orig & CC_kids, 'Edu_c'] = 1
purp_filt.loc[CCKindy_dest & CC_kids, 'Edu_c'] = 1

# purp_filt.loc[educ_O_D & non_all_student, 'Edu_non'] = 1

### Shop Flag Definition
As it is defined below, Shop place flag represents the records which either their **origin place** or **destination place** is within the places: 
* Shop or supermarket, 
* Restaurant, cafÃ©, or food outlet, 
* Petrol station 

In [ ]:
"""Shopping."""
shop_orig = purp_filt.ORIGPLACE == 'Shop or supermarket' 
shop_dest = purp_filt.DESTPLACE == 'Shop or supermarket'
food_orig = purp_filt.ORIGPLACE == 'Restaurant, cafÃ©, or food outlet'
food_dest = purp_filt.DESTPLACE == 'Restaurant, cafÃ©, or food outlet'
fuel_orig = purp_filt.ORIGPLACE == 'Petrol station'
fuel_dest = purp_filt.DESTPLACE == 'Petrol station'

purp_filt.loc[shop_orig | shop_dest | food_orig | food_dest | fuel_dest | fuel_orig , 'shop'] = 1
purp_filt.loc[shop_orig | food_orig, 'Orig_Type'] = 'Shopping'
purp_filt.loc[shop_dest | food_dest, 'Dest_Type'] = 'Shopping'
purp_filt.loc[fuel_orig , 'Orig_Type'] = 'Fuel'
purp_filt.loc[fuel_dest , 'Dest_Type'] = 'Fuel'
del shop_orig, shop_dest, food_orig, food_dest, fuel_dest, fuel_orig

### Escort Flag Definition
Escort flag represents the records where either the origin-purpose or the destination-purpose is transporting a passenger/s or pick up or drop off a passenger or just-accompanying-someone.

In [ ]:
#Escort Flag
escort_purp_orig_1_flag = purp_filt.ORIGPURP == 'Transporting a passenger/s'
escort_purp_dest_1_flag = purp_filt.DESTPURP == 'Transporting a passenger/s'
escort_purp_orig_2_flag = purp_filt.ORIGPURP == 'Pick up or drop off a passenger'
escort_purp_dest_2_flag = purp_filt.DESTPURP == 'Pick up or drop off a passenger'
escort_purp_orig_3_flag = purp_filt.ORIGPURP == 'Just accompanying someone'
escort_purp_dest_3_flag = purp_filt.DESTPURP == 'Just accompanying someone'

purp_filt.loc[escort_purp_orig_1_flag | escort_purp_orig_2_flag | escort_purp_dest_1_flag | escort_purp_dest_2_flag | escort_purp_orig_3_flag | escort_purp_dest_3_flag, 'escort'] = 1

del escort_purp_orig_1_flag, escort_purp_orig_2_flag, escort_purp_dest_1_flag, escort_purp_dest_2_flag
del escort_purp_orig_3_flag, escort_purp_dest_3_flag

### Other Flag Definition

Others flag represents the records where either the origin-place or destination-place is someone else's home or other-place-(please-specify).

In [ ]:
#Other Flag
other_home_orig = purp_filt.ORIGPLACE == "Someone else's home"
other_home_dest = purp_filt.DESTPLACE == "Someone else's home"
purp_filt.loc[other_home_orig | other_home_dest, 'others home'] = 1
other_place_orig = purp_filt.ORIGPLACE == "Other place (please specify)"
other_place_dest = purp_filt.DESTPLACE == "Other place (please specify)"
purp_filt.loc[other_home_orig | other_home_dest | other_place_orig | other_place_dest, 'other'] = 1
purp_filt.loc[other_home_orig, 'Orig_Type'] = 'Other Home'
purp_filt.loc[other_place_orig , 'Orig_Type'] = 'Other Place'
purp_filt.loc[other_home_dest, 'Dest_Type'] = 'Other Home'
purp_filt.loc[other_place_dest, 'Dest_Type'] = 'Other Place'
del other_home_orig, other_home_dest, other_place_orig, other_place_dest
purp_filt.fillna(0, inplace = True)

## 1. Home Based Trips
Home based trips are the ones where their home based flags are true or in other words, their origin or destination places are My Home or their origin or destination purposes are At Home or Go Home.

These trips based on their other conditions categorized as the following categories:
* HBWW
* HBWB
* HBEp
* HBEs
* HBEt
* HBEc
* HBEe
* HBS
* HBOr
* HBOe
* HBO

### 1.1. Home Based Work (HBWW and HBWB)
A home-based work trip defines as a home-based trip which either its origin or destination purpose is "Work (my workplace)" or "Work (other work reason)", in addition to the condition that the person should be a worker either white or blue.

A person is considered as worker when:
* their WORKSTATUS is WorkFullTime or WorkPartTime; or
* Or their age are greater than 14, and have made at least a trip with the destination place is My Workplace and the duration is greater than 60. (Correction)

Being white or blue is determined based on the Collar flags in Person Table and ANZSCO_adjustment.csv. 

**Note**: If a worker (as it is defined in the correction part) has a person studying value equalls to "Primary" or his/her WORKSTATUS is "retired" or his/her age is less than 15, he/she recieves Collar tag of Nan or 0 and he/she is considered as Worker_White in the correction part. 

In [ ]:
"""Trip Purpose Calculator"""
purp_filt['Trip_Purpose'] = 0

#HBW - White & Blue
w_collar_flag = purp_filt.Worker_White == 1
b_collar_flag = purp_filt.Worker_Blue == 1
hb_flag = purp_filt.Home_based == 1
work_flag = purp_filt.work == 1
purp_filt.loc[w_collar_flag & hb_flag & work_flag, 'Trip_Purpose'] = 'HBWW'
done_flag = purp_filt.Trip_Purpose == 0
purp_filt.loc[b_collar_flag & hb_flag & work_flag, 'Trip_Purpose'] = 'HBWB'

del w_collar_flag, b_collar_flag, work_flag, hb_flag
purp_filt.Trip_Purpose.value_counts()

### 1.1. Home Based Education (HBEp, HBEs, HBEs, HBEt, HBEc)
Home based Education trip generally defines as a home based trip which either its origin or destination place is one of the education places, in addition to the condition that the person should be student and the trip purpose be Education.

A person is considered as student when in Person table his/her STUDYING value is primary, secondary, tertiaryFullTime, tertiaryPartTime or his/her MAINACT is Childcare/Kindergarten (or as modified later in the code his/her STUDYING value is CC/Kindy)

**Note**: For HBEc, none of the records has Education purpose on none of their ends.

In [ ]:
done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1
edu_purp_flag = purp_filt["Edu_purp"] == 1
edu_p_flag = purp_filt['Edu_p'] == 1
edu_s_flag = purp_filt['Edu_s'] == 1
edu_t_flag = purp_filt['Edu_t'] == 1
edu_c_flag = purp_filt['Edu_c'] == 1

purp_filt.loc[done_flag & hb_flag & edu_purp_flag & edu_p_flag, 'Trip_Purpose'] = 'HBEp'
purp_filt.loc[done_flag & hb_flag & edu_purp_flag & edu_s_flag, 'Trip_Purpose'] = 'HBEs'
purp_filt.loc[done_flag & hb_flag & edu_purp_flag & edu_t_flag, 'Trip_Purpose'] = 'HBEt'
purp_filt.loc[done_flag & hb_flag  & edu_c_flag, 'Trip_Purpose'] = 'HBEc'    
## Modification: edu_purp_flag should be added to the conditions for the HBEc catagory

In [ ]:
purp_filt.Trip_Purpose.value_counts()

* #### Check the necessity of considering My Campus as education places for HBEp
To check this need, we calculate the number of trips by considering the HBEp conditions, specifically setting My Campus as Education plac in defining "Edu_p" (in Education Flag definitions section). As it is shown below, the age of all of the people with these conditions are between 5 and 12. Thus, we consider this place too in Edu_p definition.

In [ ]:
purp_filt[purp_filt['Trip_Purpose']== 'HBEp']['AGE']

* #### Check the necessity of considering My Campus as education places for HBEs
To check this need, we calculate the number of trips by considering the HBEs conditions, specifically setting My Campus as Education plac in defining "Edu_s" (in Education Flag definitions section). As it is shown below, the age of all of the people with these conditions are between 14 and 18. Thus, we consider this place too in Edu_s definition.

In [ ]:
purp_filt[purp_filt['Trip_Purpose']== 'HBEs']['AGE']

In [ ]:
print("Number unique PERSIDs with HBEc trip purpose:")
HBEC_unique_pers = len(purp_filt[purp_filt['Trip_Purpose'] == 'HBEc'].PERSID.unique())
print(HBEC_unique_pers)

print("Number unique TRIPIDs with HBEc trip purpose:")
HBEC_unique_trip = len(purp_filt[purp_filt['Trip_Purpose'] == 'HBEc'].TRIPID.unique())
print(HBEC_unique_trip)


In [ ]:
"""Reclassify Escorted School trips."""
done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1
hb_o_flag = purp_filt.HB_Orig == 1
hb_d_flag = purp_filt.HB_Dest == 1
edu_cps_flag = purp_filt['Edu_cps_place'] == 1
escort_flag = purp_filt.escort == 1
CC_student = purp_filt.Student_CC == 1
prim_student = purp_filt.Student_Prim == 1
sec_student = purp_filt.Student_Sec == 1
young_student = purp_filt.Student_Young == 1
dur_u15_a = purp_filt.Dur == 'u15'
dur_o60_a = purp_filt.Dur == 'o60'

purp_filt.loc[done_flag & hb_o_flag & edu_cps_flag & CC_student & dur_o60_a, 'Trip_Purpose'] = 'HBEc'
purp_filt.loc[done_flag & hb_o_flag & edu_cps_flag & prim_student & dur_o60_a, 'Trip_Purpose'] = 'HBEp'
purp_filt.loc[done_flag & hb_d_flag & edu_cps_flag & CC_student & dur_o60_a, 'Trip_Purpose'] = 'HBEc'
purp_filt.loc[done_flag & hb_d_flag & edu_cps_flag & prim_student & dur_o60_a, 'Trip_Purpose'] = 'HBEp'

purp_filt.loc[done_flag & hb_o_flag & edu_cps_flag & sec_student & dur_o60_a, 'Trip_Purpose'] = 'HBEs'
purp_filt.loc[done_flag & hb_d_flag & edu_cps_flag & sec_student & dur_o60_a, 'Trip_Purpose'] = 'HBEs'

print("Number of unique PERSIDs with HBEc trip purpose after correction:")
HBEC_unique_pers = len(purp_filt[purp_filt['Trip_Purpose'] == 'HBEc'].PERSID.unique())
print(HBEC_unique_pers)

print("Number of unique TRIPIDs with HBEc trip purpose after correction:")
HBEC_unique_trip = len(purp_filt[purp_filt['Trip_Purpose'] == 'HBEc'].TRIPID.unique())
print(HBEC_unique_trip)

hbec_trips = purp_filt['Trip_Purpose'] == 'HBEc'
edu_purp_flag = purp_filt["Edu_purp"] == 1
HBEC_education_end_purpose = len(purp_filt.loc[hbec_trips & edu_purp_flag].Trip_Purpose.value_counts())
print("Number of HBEc trips with Education End purpose:")
print(HBEC_education_end_purpose)

del CC_student, prim_student, young_student, dur_u15_a, dur_o60_a

**Note**: As it is showed above, by performing correction, the number of unique PERSIDs do not increased, but the 3 new HBEc trips ID are added for HBEc trips.

In [ ]:
purp_filt.Trip_Purpose.value_counts()

In [ ]:
"""Education Escort Trips."""
done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1
hb_o_flag = purp_filt.HB_Orig == 1
hb_d_flag = purp_filt.HB_Dest == 1
driver_flag = purp_filt.MAINMODE == 'Car driver'
escort_flag = purp_filt.escort == 1
non_student = purp_filt.Student_Young != 1#################################
young_student = purp_filt.Student_Young == 1
dur_u15_a = purp_filt.Dur == 'u15'
dur_o15_a = purp_filt.Dur == 'o15'

edu_cps_flag = purp_filt['Edu_cps_place'] == 1
# purp_filt.loc[done_flag & hb_flag & edu_cps_flag & driver_flag & escort_flag, 'Trip_Purpose'] = 'HBEe' #
purp_filt.loc[done_flag & hb_flag & edu_cps_flag & escort_flag, 'Trip_Purpose'] = 'HBEe' #

print("Number of trip records where the duration is below 15:")
hbee_u15 = purp_filt[done_flag & hb_flag & edu_cps_flag & escort_flag & dur_u15_a].Trip_Purpose.count() #
print(hbee_u15)

print("Number of trip records where the duration is above 15:")
hbee_o15 = purp_filt[done_flag & hb_flag & edu_cps_flag & escort_flag & dur_o15_a].Trip_Purpose.count() #
print(hbee_o15)

done_flag = purp_filt.Trip_Purpose == 0

purp_filt.Trip_Purpose.value_counts()

# Omitted Correction:
# purp_filt.loc[done_flag & hb_flag & edu_cps_flag & driver_flag & edu_purp_flag , 'Trip_Purpose'] = 'HBEe'

In [ ]:
# Correction 1:
purp_filt.loc[done_flag & hb_flag & edu_cps_flag & non_student & escort_flag , 'Trip_Purpose'] = 'HBEe'
purp_filt.Trip_Purpose.value_counts()

In [ ]:
# Correction 2:
purp_filt.loc[done_flag & hb_o_flag & edu_cps_flag & young_student & escort_flag & dur_u15_a , 'Trip_Purpose'] = 'HBEe'
purp_filt.Trip_Purpose.value_counts()

In [ ]:
# Correction 3:
purp_filt.loc[done_flag & hb_d_flag & edu_cps_flag & young_student & escort_flag, 'Trip_Purpose'] = 'HBEe'

del non_student, young_student, dur_u15_a, escort_flag, hb_o_flag, hb_d_flag,hb_flag

In [ ]:
purp_filt.Trip_Purpose.value_counts()

### 1.3. Home Based Shopping (HBS)

In [ ]:
"""HBS - Shopping"""
done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1
shop_flag = purp_filt.shop == 1

# purp_filt.loc[done_flag & hb_flag & shop_flag, 'Trip_Purpose'] = 'HBS' 
# del done_flag, hb_flag, shop_flag
## Modification: the last two lines become comment since they check the places not the purposes for HBS trips. The next
## lines are added.

eat_flag_orig = purp_filt.ORIGPURP == 'Eat/drink'
eat_flag_dest = purp_filt.DESTPURP == 'Eat/drink'
shop_flag_orig = purp_filt.ORIGPURP == 'Shopping'
shop_flag_dest = purp_filt.DESTPURP == 'Shopping'
fuel_flag_orig = purp_filt.ORIGPURP == 'Refuel my vehicle'
fuel_flag_dest = purp_filt.DESTPURP == 'Refuel my vehicle'

purp_filt.loc[eat_flag_orig | eat_flag_dest | shop_flag_orig | shop_flag_dest | fuel_flag_orig | fuel_flag_dest, 'shop_purp'] = 1
shop_purp_flag = purp_filt.shop_purp == 1
purp_filt.loc[shop_purp_flag & shop_flag & hb_flag & done_flag, "Trip_Purpose"] = 'HBS'


In [ ]:
purp_filt.Trip_Purpose.value_counts()

### 1.4. Home Based Recreation (HBOr)

In [ ]:
"""#HBO Recreation"""
rec_orig = purp_filt.ORIGPURP == 'Recreation/leisure activity'
rec_dest = purp_filt.DESTPURP == 'Recreation/leisure activity'
done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1

purp_filt.loc[done_flag & hb_flag & rec_orig, 'Trip_Purpose'] = 'HBOr'
purp_filt.loc[done_flag & hb_flag & rec_dest, 'Trip_Purpose'] = 'HBOr'
del rec_orig, rec_dest, hb_flag, done_flag

In [ ]:
purp_filt.Trip_Purpose.value_counts()

### 1.5. Home Based Other Escort & Other (HBOe and HBO)

In [ ]:
"""HBO - Escort & Other"""
done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1
escort_flag = purp_filt.escort == 1
other_flag = purp_filt.other == 1
purp_filt.loc[done_flag & hb_flag & other_flag & escort_flag, 'Trip_Purpose'] = 'HBOe'

done_flag = purp_filt.Trip_Purpose == 0
purp_filt.loc[done_flag & hb_flag & other_flag, 'Trip_Purpose'] = 'HBO'
del done_flag, hb_flag, escort_flag, other_flag

In [ ]:
purp_filt.Trip_Purpose.value_counts()

### 1.6. Home Based Trips - Final Allocation

In [ ]:
#Non-Collared_Work
done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1
work_flag = purp_filt.work == 1



age_filter = purp_filt.AGE > 14
dest_accompy = purp_filt.DESTPURP != 'Just accompanying someone'
orig_accompy = purp_filt.ORIGPURP != 'Just accompanying someone'

dest_drop = purp_filt.DESTPURP != 'Pick up or drop off a passenger'
orig_drop = purp_filt.ORIGPURP != 'Pick up or drop off a passenger'

dest_rec = purp_filt.DESTPURP != 'Recreation/leisure activity'
orig_rec = purp_filt.ORIGPURP != 'Recreation/leisure activity'
dest_deliv = purp_filt.DESTPURP != 'Pickup/deliver something (not work-related)'
orig_deliv = purp_filt.ORIGPURP != 'Pickup/deliver something (not work-related)'
dest_social = purp_filt.DESTPURP != 'Social Visit'
orig_social = purp_filt.ORIGPURP != 'Social Visit'
dest_shop = purp_filt.DESTPURP != 'Shopping'
orig_shop = purp_filt.ORIGPURP != 'Shopping'


purp_filt.loc[done_flag & hb_flag & work_flag & age_filter & dest_accompy & orig_accompy & dest_drop & orig_drop & dest_rec & orig_rec & dest_deliv & orig_deliv & dest_social & orig_social & dest_shop & orig_shop, 'Trip_Purpose'] = 'HBWW'
#purp_filt.loc[done_flag & hb_flag & work_flag, 'Trip_Purpose'] = 'HBWW'

del age_filter, dest_accompy, orig_accompy, dest_drop, orig_drop, dest_rec, orig_rec, dest_deliv, orig_deliv, dest_social, orig_social, dest_shop, orig_shop


#HomeBaseHome LMAO xD
done_flag = purp_filt.Trip_Purpose == 0
home_orig = purp_filt.ORIGPLACE == 'My home'
home_dest = purp_filt.DESTPLACE == 'My home'
purp_filt.loc[done_flag & home_orig & home_dest, 'Trip_Purpose'] = 'HBOr' #now HBOr
del done_flag, home_orig, home_dest
#print(purp_filt['Trip_Purpose'].value_counts())

#Home Based Escorting
done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1
escort_flag = purp_filt.escort == 1
purp_filt.loc[done_flag & hb_flag & escort_flag, 'Trip_Purpose'] = 'HBOe'
del done_flag, hb_flag, escort_flag


#print(purp_filt['Trip_Purpose'].value_counts())

#Home Based Other - Remainder
done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1
purp_filt.loc[done_flag & hb_flag, 'Trip_Purpose'] = 'HBO'

del done_flag, hb_flag

In [ ]:
purp_filt.Trip_Purpose.value_counts()

### Report HB Trips

In [ ]:
hb_report = purp_filt.copy()
hb_report = hb_report.groupby("Trip_Purpose").Trip_Purpose.count()
hb_report = pd.DataFrame(hb_report).swapaxes("index", "columns").rename(columns={0:'Not Allocated'}, index={'Trip_Purpose': 'number of trips'})
hb_report
# purp_filt.Trip_Purpose.value_counts().keys()[1]

## 2. Non-Home Based Trips
Non-Home based trips are the ones where their home based flags are false .

These trips based on their other conditions categorized as the following categories:
* NHBW
* WBW
* NHBE
* NHBO
* NHBS

### 2.1. Non-Home Based Work and Work Base Work (NHBW and WBW)

**Note**: For NHBW, the trip direction is important and trips which their destinations purpose are work related are only considered.

In [ ]:
#NHB - Education, Work, Shop and Other
done_flag = purp_filt.Trip_Purpose == 0
non_hb_flag = purp_filt.Home_based != 1

#Work Purpose
work_d_purp = purp_filt.DESTPURP == 'Work (my workplace)'
work_d_purp2 = purp_filt.DESTPURP == 'Work (other work reason)'
work_o_purp = purp_filt.ORIGPURP == 'Work (my workplace)'
work_o_purp2 = purp_filt.ORIGPURP == 'Work (other work reason)'

#WorkTrip Directionality

purp_filt.loc[work_o_purp | work_o_purp2, 'work_o'] = 1
purp_filt.loc[work_d_purp | work_d_purp2, 'work_d'] = 1
work_d_flag = purp_filt.work_d == 1
work_o_flag = purp_filt.work_o == 1

purp_filt.loc[non_hb_flag & work_o_flag, 'Work_Direction'] = 'Work2Attr'
purp_filt.loc[non_hb_flag & work_d_flag, 'Work_Direction'] = 'Attr2Work'
purp_filt.loc[done_flag & non_hb_flag & work_d_flag & work_o_flag, 'Trip_Purpose'] = 'WBW'

done_flag = purp_filt.Trip_Purpose == 0 # added this line, since the order of WBW and NHBW is changed.

purp_filt.loc[done_flag & non_hb_flag & work_d_flag, 'Trip_Purpose'] = 'NHBW'

del work_d_purp, work_d_purp2, work_d_flag, work_o_purp, work_o_purp2, work_o_flag

In [ ]:
purp_filt.Trip_Purpose.value_counts()

### 2.2. Non-Home Based Education (NHBE)

**NHBE Flowchart**
<img src="../Media/NHBE.jpg">

In [ ]:
#education NHB
done_flag = purp_filt.Trip_Purpose == 0
educ_purp_d = purp_filt.DESTPURP == 'Education'
primsec_dest = purp_filt.DESTPLACE == 'Primary or secondary school' 
CCKindy_dest = purp_filt.DESTPLACE == 'Childcare or kindergarten'
tert_dest = purp_filt.DESTPLACE == 'My campus' 
other_ed_dest = purp_filt.DESTPLACE == 'Other study-related place' 
tert_student = purp_filt.Student_Tert == 1
prim_student = purp_filt.Student_Prim == 1
sec_student = purp_filt.Student_Sec == 1 
escort_flag = purp_filt.escort == 1

# pickdrop_1 = purp_filt.DESTPURP == 'Pick up or drop off a passenger'
# pickdrop_2 = purp_filt.DESTPURP == 'Transporting a passenger/s'
# pickdrop_3 = purp_filt.DESTPURP == 'Just accompanying someone'
edu_dest_flagger = purp_filt.Dest_Type == 'Education' 
edu_orig_flagger = purp_filt.Orig_Type == 'Education'
edu_place_flagger = edu_dest_flagger | edu_orig_flagger

purp_filt.loc[primsec_dest | CCKindy_dest | tert_dest | other_ed_dest, 'edu_dest'] = 1
edu_dest_flag = purp_filt['edu_dest'] == 1

purp_filt.loc[done_flag & non_hb_flag & educ_purp_d & edu_dest_flag & prim_student, 'Trip_Purpose'] = 'NHBE'
purp_filt.loc[done_flag & non_hb_flag & educ_purp_d & edu_dest_flag & sec_student, 'Trip_Purpose'] = 'NHBE'
purp_filt.Trip_Purpose.value_counts()

In [ ]:
# purp_filt.loc[done_flag & non_hb_flag & edu_dest_flagger & pickdrop_1, 'Trip_Purpose'] = 'NHBE'
# purp_filt.loc[done_flag & non_hb_flag & edu_dest_flagger & pickdrop_2, 'Trip_Purpose'] = 'NHBE'
# purp_filt.loc[done_flag & non_hb_flag & edu_dest_flagger & pickdrop_3, 'Trip_Purpose'] = 'NHBE'
purp_filt.loc[done_flag & non_hb_flag & edu_place_flagger & escort_flag, 'Trip_Purpose'] = 'NHBE'

## Modification: the pickdrop_1,2,3 represent escort flag for destination. Also, I don't understand why origin
## escort flag and edu_origin_flagger is not considered. If origins get considered too, we can write the next two lines instead of the last 
## three lines.

# escort_flag = purp_filt.escort == 1
# purp_filt.loc[done_flag & non_hb_flag & edu_dest_flagger & escort_flag, 'Trip_Purpose'] = 'NHBE'

purp_filt.loc[done_flag & non_hb_flag & educ_purp_d & edu_dest_flag & tert_student, 'Trip_Purpose'] = 'NHBO' #Not NHBE

del done_flag, educ_purp_d, primsec_dest, CCKindy_dest, tert_dest, other_ed_dest, tert_student, prim_student, sec_student, edu_dest_flag

In [ ]:
purp_filt.Trip_Purpose.value_counts()

### 2.3. Non-Home Based Shopping (NHBS)

**NHBS Flowchart**
<img src="../Media/NHBS.jpg">

In [ ]:
#shopping
non_hb_flag = purp_filt.Home_based != 1
shop_dest = purp_filt.DESTPLACE == 'Shop or supermarket'
food_dest = purp_filt.DESTPLACE == 'Restaurant, cafÃ©, or food outlet'
fuel_dest = purp_filt.DESTPLACE == 'Petrol station'
purp_filt.loc[shop_dest | food_dest | fuel_dest, 'shop_dest'] = 1
done_flag = purp_filt.Trip_Purpose == 0
shop_d_flag = purp_filt.shop_dest == 1
purp_filt.loc[done_flag & non_hb_flag & shop_d_flag, 'Trip_Purpose'] = 'NHBS'
del done_flag, shop_d_flag, shop_dest, food_dest, fuel_dest

In [ ]:
purp_filt.Trip_Purpose.value_counts()

### 2.4. Non-Home Based - Final Allocation

In [ ]:
done_flag = purp_filt.Trip_Purpose == 0
other_flag = purp_filt.other == 1
purp_filt.loc[done_flag & non_hb_flag & other_flag, 'Trip_Purpose'] = 'NHBO'
del done_flag, non_hb_flag,  other_flag
purp_filt.Trip_Purpose.value_counts()

In [ ]:
#Non-Home based Other - Remainder
done_flag = purp_filt.Trip_Purpose == 0
non_hb_flag = purp_filt.Home_based != 1
purp_filt.loc[done_flag & non_hb_flag, 'Trip_Purpose'] = 'NHBO'
del done_flag, non_hb_flag

In [ ]:
purp_filt.Trip_Purpose.value_counts()

## Trip Purpose HTML

In [ ]:
trip_purp_table = purp_filt[['TRIPID', 'ORIGPLACE', 'DESTPLACE', 'ORIGPURP', 'DESTPURP', 'Trip_Purpose', 'TRIPWGT19']]
pivot_ui(trip_purp_table, vals=['TRIPID', 'TRIPWGT19'], aggregatorName=['Count', 'Sum'], outfile_path='Trips_purpose_pivot.html')

In [ ]:
trip = purp_filt
del purp_filt

In [ ]:
"""Direction Flag."""
trip_dir = trip.copy()
trip_dir['Direction'] = 'OD'
p2a_flag = trip_dir.HB_Orig == 1
trip_dir.loc[p2a_flag, 'Direction'] = 'P2A'
a2p_flag = trip_dir.HB_Dest == 1
trip_dir.loc[a2p_flag, 'Direction'] = 'A2P'
del p2a_flag, a2p_flag

od_flag = trip_dir.Direction == 'OD'
trip_dir.loc[od_flag, 'PROD_SA1'] = trip_dir.loc[od_flag, 'ORIGSA1_2016'] 
trip_dir.loc[od_flag, 'ATTR_SA1'] = trip_dir.loc[od_flag, 'DESTSA1_2016']
trip_dir.loc[od_flag, 'PROD_LONG'] = trip_dir.loc[od_flag, 'ORIGLONG'] 
trip_dir.loc[od_flag, 'PROD_LAT'] = trip_dir.loc[od_flag, 'ORIGLAT'] 
trip_dir.loc[od_flag, 'ATTR_LONG'] = trip_dir.loc[od_flag, 'DESTLONG']
trip_dir.loc[od_flag, 'ATTR_LAT'] = trip_dir.loc[od_flag, 'DESTLAT'] 
del od_flag
p2a_flag = trip_dir.Direction == 'P2A'
trip_dir.loc[p2a_flag, 'PROD_SA1'] = trip_dir.loc[p2a_flag, 'ORIGSA1_2016'] 
trip_dir.loc[p2a_flag, 'ATTR_SA1'] = trip_dir.loc[p2a_flag, 'DESTSA1_2016'] 
trip_dir.loc[p2a_flag, 'PROD_LONG'] = trip_dir.loc[p2a_flag, 'ORIGLONG'] 
trip_dir.loc[p2a_flag, 'PROD_LAT'] = trip_dir.loc[p2a_flag, 'ORIGLAT'] 
trip_dir.loc[p2a_flag, 'ATTR_LONG'] = trip_dir.loc[p2a_flag, 'DESTLONG']
trip_dir.loc[p2a_flag, 'ATTR_LAT'] = trip_dir.loc[p2a_flag, 'DESTLAT'] 
del p2a_flag
a2p_flag = trip_dir.Direction == 'A2P'
trip_dir.loc[a2p_flag, 'PROD_SA1'] = trip_dir.loc[a2p_flag, 'DESTSA1_2016'] 
trip_dir.loc[a2p_flag, 'ATTR_SA1'] = trip_dir.loc[a2p_flag, 'ORIGSA1_2016'] 
trip_dir.loc[a2p_flag, 'PROD_LONG'] = trip_dir.loc[a2p_flag, 'DESTLONG'] 
trip_dir.loc[a2p_flag, 'PROD_LAT'] = trip_dir.loc[a2p_flag, 'DESTLAT'] 
trip_dir.loc[a2p_flag, 'ATTR_LONG'] = trip_dir.loc[a2p_flag, 'ORIGLONG']
trip_dir.loc[a2p_flag, 'ATTR_LAT'] = trip_dir.loc[a2p_flag, 'ORIGLAT'] 
del a2p_flag

trip = trip_dir
del trip_dir

"""Prod/Attr SA2-4,LGA Zones"""
zone_filt = trip.copy()
zone_filt = pd.merge(zone_filt, ASGS.set_index('SA1_7DIGITCODE_2016')[['SA2_NAME_2016','SA3_NAME_2016', 'SA4_NAME_2016','LGA_NAME_2016']], left_on = 'PROD_SA1', right_index = True, how = 'left').rename(columns={'SA2_NAME_2016': "PROD_SA2", 'SA3_NAME_2016': 'PROD_SA3', 'SA4_NAME_2016':'PROD_SA4', 'LGA_NAME_2016':'PROD_LGA'})
zone_filt = pd.merge(zone_filt, ASGS.set_index('SA1_7DIGITCODE_2016')[['SA2_NAME_2016','SA3_NAME_2016', 'SA4_NAME_2016','LGA_NAME_2016']], left_on = 'ATTR_SA1', right_index = True, how = 'left').rename(columns={'SA2_NAME_2016': "ATTR_SA2", 'SA3_NAME_2016': 'ATTR_SA3', 'SA4_NAME_2016':'ATTR_SA4', 'LGA_NAME_2016':'ATTR_LGA'})

zone_filt = pd.merge(zone_filt, ASGS.set_index('SA1_7DIGITCODE_2016')[['SA2_NAME_2016','SA3_NAME_2016', 'SA4_NAME_2016','LGA_NAME_2016']], left_on = 'ORIGSA1_2016', right_index = True, how = 'left').rename(columns={'SA2_NAME_2016': "ORIG_SA2", 'SA3_NAME_2016': 'ORIG_SA3', 'SA4_NAME_2016':'ORIG_SA4', 'LGA_NAME_2016':'ORIG_LGA'})
zone_filt = pd.merge(zone_filt, ASGS.set_index('SA1_7DIGITCODE_2016')[['SA2_NAME_2016','SA3_NAME_2016', 'SA4_NAME_2016','LGA_NAME_2016']], left_on = 'DESTSA1_2016', right_index = True, how = 'left').rename(columns={'SA2_NAME_2016': "DEST_SA2", 'SA3_NAME_2016': 'DEST_SA3', 'SA4_NAME_2016':'DEST_SA4', 'LGA_NAME_2016':'DEST_LGA'})


print('Number of Trip Records in Original: {}'.format(len(TRIPS.axes[0])))
print('Number of Trip Records: {}'.format(len(zone_filt.axes[0])))

trip = zone_filt
del zone_filt

In [ ]:
%%time
"""Time Period Adjustment."""
time_filt = trip.copy()

time_filt['MidTime'] = (((time_filt['STARTIME'] + (time_filt['TRAVTIME']/2)).round(1) + 4*60)/60) %24
time_filt['MidTimeHr'] = time_filt['MidTime'].apply(np.floor)
time_filt['StartHr'] = (((time_filt['STARTIME'] + 4*60)/60) %24).apply(np.floor)
time_filt['EndHr'] = (((time_filt['STARTIME'] + time_filt['TRAVTIME'] + 4*60)/60) %24).apply(np.floor)

time_filt['MidTimeHr'].plot.hist(bins=48,alpha =0.5)
time_filt['StartHr'].plot.hist(bins=48,alpha =0.5)
time_filt['EndHr'].plot.hist(bins=48,alpha =0.5)
trip = time_filt
del time_filt


## Mode Adjustments
In this section, we assing MODELMODE and PT_access for each trip. To reach this goal, we follow the following steps successively:

1. Consider left modes as right modes (MAINMODE, MODE1, MODE2, ..., MODE8):
    Truck driver, Motorcycle driver, Other method -----> Car driver
    Truck passenger, Motorcycle passenger, Taxi, Taxi or Uber, Taxi or ride share e.g. Uber, Uber / Other Ride Share -----> Car passenger
    Mobility scooter -----> Bicycle
    
2. Define MODELMODE according to the modified MAINMODE (in step 1) as Car driver, Car passenger, Bicycle and Walking. MODELNAME for other MAINMODEs will be defined in the next steps. Thus, if MAINMODE of a trip is within the above group of modes, its MODELNAME is defined, otherwise its MODELNAME is not defined yet. 

3. Consider left modes as right modes (MAINMODE, MODE1, MODE2, ..., MODE8):
    Public Bus, Charter/Courtesy/Other bus -----> Public bus
    School bus (private/chartered), School bus (with route number) -----> School bus
    
4. Based on the number of stops in a trip and its MODE1, MODE2, ..., MODE8, we define MODELNAME. Some general points about this allocation is as stated below:
    * Trips with 0 stop (only having MODE1) and without MODELNAME get this property equal to their MAINMODE. In additon, their PT_access is Walk2PT.

    * Trips with n>=1 stop (only having MODE1, MODE2, ... MODE_n+1) and without MODELNAME get this property equal to their MAINMODE. In additon, their PT_access is:
     - if their MODE1 is within walking, car driver or car passenger and their last MODE (MODE_n+1) is not within this group : PT_access is defined based on the MODE1.
 
     - if their MODE1 is within walking, car driver or car passenger and their last MODE (MODE_n+1) is within this group too: PT_access is defined based on the last MODE (MODE_n+1).
 
     - if their MODE1 is not within walking, car driver or car passenger and their last MODE (MODE_n+1) is within this group: PT_access is defined based on the last MODE (MODE_n+1).
 
     - if their MODE1 is not within walking, car driver or car passenger and their last MODE (MODE_n+1) is not within this group too: PT_access is defined as Walk2PT.
 
**IMPORTANT NOTE**: For trips with number of stops equal or greater than 2 stops (having at least MODE1, MODE2, MODE3), the trip direction as P2A, A2P and OD is considered too to define PT_access. For example, for a trips with two stops, if MODE1 = car driver and MODE3 (last MODE) != (Car driver | car passenger | walking), we cannot say this trip PT_access is not defined based on its MODE1 as "PnR" since its direction is A2P (not P2A or OD) and it will be defined as Walk2PT at the end. 

**Question 1: Is is correct to consider direction? If so, why shouldn't we consider it for trips with one stop too (having MODE1 and MODE2)?**

**Question 2: Why do we consider only first mode and last mode for defining PT_access?**

**Question 3: PT_access is not defined for trips with MAINMODE (not MODE1, MODE2, ...) of Car driver, Car passenger, Bicycle and Walking. What PT_access should we assign to them?**
    

In [ ]:
%%time
"""Mode Adjustments."""
mode_filt = trip.copy()
mode_filt['MODELMODE'] = 0

#Mode Flags


mode_filt.replace({'Truck driver': 'Car driver','Truck passenger': 'Car passenger', 'Motorcycle driver': 'Car driver', 'Motorcycle passenger': 'Car passenger','Mobility scooter': 'Bicycle', 'Taxi': 'Car passenger', 'Taxi or Uber': 'Car passenger','Taxi or ride share e.g. Uber' : 'Car passenger', 'Uber / Other Ride Share' : 'Car passenger', 'Other method': 'Car driver'}, inplace = True)

car_driver = mode_filt.MAINMODE == 'Car driver'
car_pass = mode_filt.MAINMODE == 'Car passenger'
single_mode = mode_filt.MODE2 == 0

walk = mode_filt.MAINMODE == 'Walking'
bicycle = mode_filt.MAINMODE == 'Bicycle'
motorcycle_driver = mode_filt.MAINMODE == 'Motorcycle driver'
motorcycle_pass = mode_filt.MAINMODE == 'Motorcycle passenger'
truck_driver = mode_filt.MAINMODE == 'Truck driver'
truck_pass = mode_filt.MAINMODE == 'Truck passenger'
mob_scoot = mode_filt.MAINMODE == 'Mobility scooter'
taxi = mode_filt.MAINMODE == 'Taxi'
uber = mode_filt.MAINMODE == 'Taxi or Uber'
ride_share = mode_filt.MAINMODE == 'Taxi or ride share e.g. Uber'

#single modes
mode_filt.loc[car_driver, 'MODELMODE'] = 'Car driver'
mode_filt.loc[car_pass, 'MODELMODE'] = 'Car passenger'
mode_filt.loc[walk, 'MODELMODE'] = 'Walking'
mode_filt.loc[bicycle, 'MODELMODE'] = 'Bicycle'
mode_filt.loc[motorcycle_driver, 'MODELMODE'] = 'Car driver'
mode_filt.loc[motorcycle_pass, 'MODELMODE'] = 'Car passenger'
mode_filt.loc[truck_driver, 'MODELMODE'] = 'Car driver'
mode_filt.loc[truck_pass, 'MODELMODE'] = 'Car passenger'
mode_filt.loc[mob_scoot, 'MODELMODE'] = 'Bicycle'
mode_filt.loc[taxi | uber | ride_share, 'MODELMODE'] = 'Car passenger'


mode_filt.loc[truck_driver & single_mode, 'MODELMODE'] = 'Car driver'
mode_filt.loc[truck_pass & single_mode, 'MODELMODE'] = 'Car passenger'


############
# PT MODES #
############
mode_filt.replace({'Public Bus': 'Public bus','Charter/Courtesy/Other bus': 'Public bus', 'School bus (private/chartered)': 'School bus', 'School bus (with route number)': 'School bus'}, inplace = True)

#Num Stops in trip
mode_one = mode_filt.trip_stops == 0
mode_two = mode_filt.trip_stops == 1
mode_three = mode_filt.trip_stops == 2
mode_four = mode_filt.trip_stops == 3
mode_five = mode_filt.trip_stops == 4
mode_six = mode_filt.trip_stops == 5
mode_seven = mode_filt.trip_stops == 6
mode_eight = mode_filt.trip_stops == 7

#direction flag 
a_to_p = mode_filt.Direction == 'A2P'
p_to_a = mode_filt.Direction == 'P2A'
O_to_D = mode_filt.Direction == 'OD'

#PT
# mode_filt.replace({'Public Bus': 'Public bus','Charter/Courtesy/Other bus': 'Public bus', 'School bus (private/chartered)': 'School bus', 'School bus (with route number)': 'School bus'}, inplace = True)
# bus_public = mode_filt.MAINMODE == 'Public bus'
# bus_school = mode_filt.MAINMODE == 'School bus'
# train = mode_filt.MAINMODE == 'Train' 
# light_rail = mode_filt.MAINMODE == 'Light rail' 
# ferry = mode_filt.MAINMODE == 'Ferry' 
mode_filt['PT_access'] = 'None'
not_car = mode_filt.MAINMODE != 'Car driver'
not_car_p = mode_filt.MAINMODE != 'Car passenger'
##PNR KNR W2PT one 
empty = mode_filt.MODELMODE == 0
mode_filt.loc[mode_one & empty & not_car & not_car_p, 'MODELMODE'] = mode_filt.loc[mode_one & empty, 'MAINMODE']

# Next three conditions (walk1, drive1 and dropped1) are added, since these MODE1 are not considered to define PT_access
# for mode_one trips

empty_pt_ac = mode_filt.PT_access == 'None'

mode_filt.loc[mode_one & empty & empty_pt_ac & not_car & not_car_p, 'PT_access'] = 'Walk2PT'

##PNR KNR W2PT two 
empty = mode_filt.MODELMODE == 0
mode_filt.loc[mode_two & empty & not_car & not_car_p, 'MODELMODE'] = mode_filt.loc[mode_two & empty, 'MAINMODE']

walk2 = mode_filt.MODE1 == 'Walking'
drive2 = mode_filt.MODE1 == 'Car driver'
dropped2 = mode_filt.MODE1 == 'Car passenger'
mode_filt.loc[mode_two & empty & walk2 & not_car & not_car_p, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_two & empty & drive2 & not_car & not_car_p, 'PT_access'] = 'PnR'
mode_filt.loc[mode_two & empty & dropped2 & not_car & not_car_p, 'PT_access'] = 'KnR'

walk3 = mode_filt.MODE2 == 'Walking'
drive3 = mode_filt.MODE2 == 'Car driver'
dropped3 = mode_filt.MODE2 == 'Car passenger'
mode_filt.loc[mode_two & empty & walk3 & not_car & not_car_p, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_two & empty & drive3 & not_car & not_car_p, 'PT_access'] = 'PnR'
mode_filt.loc[mode_two & empty & dropped3 & not_car & not_car_p, 'PT_access'] = 'KnR'
empty_pt_ac = mode_filt.PT_access == 'None'
mode_filt.loc[mode_two & empty & empty_pt_ac & not_car & not_car_p, 'PT_access'] = 'Walk2PT'

In [ ]:
##PNR KNR W2PT three 
empty = mode_filt.MODELMODE == 0
mode_filt.loc[mode_three & empty, 'MODELMODE'] = mode_filt.loc[mode_three & empty, 'MAINMODE']

walk2 = mode_filt.MODE1 == 'Walking'
drive2 = mode_filt.MODE1 == 'Car driver'
dropped2 = mode_filt.MODE1 == 'Car passenger'
mode_filt.loc[mode_three & empty & p_to_a & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_three & empty & p_to_a & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_three & empty & p_to_a & dropped2, 'PT_access'] = 'KnR'
mode_filt.loc[mode_three & empty & O_to_D & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_three & empty & O_to_D & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_three & empty & O_to_D & dropped2, 'PT_access'] = 'KnR'
walk3 = mode_filt.MODE3 == 'Walking'
drive3 = mode_filt.MODE3 == 'Car driver'
dropped3 = mode_filt.MODE3 == 'Car passenger'
mode_filt.loc[mode_three & empty & a_to_p & walk3, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_three & empty & a_to_p & drive3, 'PT_access'] = 'PnR'
mode_filt.loc[mode_three & empty & a_to_p & dropped3, 'PT_access'] = 'KnR'

empty_pt_ac = mode_filt.PT_access == 'None'
mode_filt.loc[mode_three & empty & empty_pt_ac, 'PT_access'] = 'Walk2PT'

In [ ]:
##PNR KNR W2PT four
empty = mode_filt.MODELMODE == 0
mode_filt.loc[mode_four & empty, 'MODELMODE'] = mode_filt.loc[mode_four & empty, 'MAINMODE']

walk2 = mode_filt.MODE1 == 'Walking'
drive2 = mode_filt.MODE1 == 'Car driver'
dropped2 = mode_filt.MODE1 == 'Car passenger'
mode_filt.loc[mode_four & empty & p_to_a & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_four & empty & p_to_a & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_four & empty & p_to_a & dropped2, 'PT_access'] = 'KnR'
mode_filt.loc[mode_four & empty & O_to_D & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_four & empty & O_to_D & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_four & empty & O_to_D & dropped2, 'PT_access'] = 'KnR'
walk3 = mode_filt.MODE4 == 'Walking'
drive3 = mode_filt.MODE4 == 'Car driver'
dropped3 = mode_filt.MODE4 == 'Car passenger'
mode_filt.loc[mode_four & empty & a_to_p & walk3, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_four & empty & a_to_p & drive3, 'PT_access'] = 'PnR'
mode_filt.loc[mode_four & empty & a_to_p & dropped3, 'PT_access'] = 'KnR'

empty_pt_ac = mode_filt.PT_access == 'None'
mode_filt.loc[mode_four & empty & empty_pt_ac, 'PT_access'] = 'Walk2PT'

##PNR KNR W2PT five
empty = mode_filt.MODELMODE == 0
mode_filt.loc[mode_five & empty, 'MODELMODE'] = mode_filt.loc[mode_five & empty, 'MAINMODE']

walk2 = mode_filt.MODE1 == 'Walking'
drive2 = mode_filt.MODE1 == 'Car driver'
dropped2 = mode_filt.MODE1 == 'Car passenger'
mode_filt.loc[mode_five & empty & p_to_a & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_five & empty & p_to_a & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_five & empty & p_to_a & dropped2, 'PT_access'] = 'KnR'
mode_filt.loc[mode_five & empty & O_to_D & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_five & empty & O_to_D & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_five & empty & O_to_D & dropped2, 'PT_access'] = 'KnR'
walk3 = mode_filt.MODE5 == 'Walking'
drive3 = mode_filt.MODE5 == 'Car driver'
dropped3 = mode_filt.MODE5 == 'Car passenger'
mode_filt.loc[mode_five & empty & a_to_p & walk3, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_five & empty & a_to_p & drive3, 'PT_access'] = 'PnR'
mode_filt.loc[mode_five & empty & a_to_p & dropped3, 'PT_access'] = 'KnR'

empty_pt_ac = mode_filt.PT_access == 'None'
mode_filt.loc[mode_five & empty & empty_pt_ac, 'PT_access'] = 'Walk2PT'

In [ ]:
##PNR KNR W2PT six
empty = mode_filt.MODELMODE == 0
mode_filt.loc[mode_six & empty, 'MODELMODE'] = mode_filt.loc[mode_six & empty, 'MAINMODE']

walk2 = mode_filt.MODE1 == 'Walking'
drive2 = mode_filt.MODE1 == 'Car driver'
dropped2 = mode_filt.MODE1 == 'Car passenger'
mode_filt.loc[mode_six & empty & p_to_a & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_six & empty & p_to_a & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_six & empty & p_to_a & dropped2, 'PT_access'] = 'KnR'
mode_filt.loc[mode_six & empty & O_to_D & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_six & empty & O_to_D & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_six & empty & O_to_D & dropped2, 'PT_access'] = 'KnR'
walk3 = mode_filt.MODE6 == 'Walking'
drive3 = mode_filt.MODE6 == 'Car driver'
dropped3 = mode_filt.MODE6 == 'Car passenger'
mode_filt.loc[mode_six & empty & a_to_p & walk3, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_six & empty & a_to_p & drive3, 'PT_access'] = 'PnR'
mode_filt.loc[mode_six & empty & a_to_p & dropped3, 'PT_access'] = 'KnR'

empty_pt_ac = mode_filt.PT_access == 'None'
mode_filt.loc[mode_six & empty & empty_pt_ac, 'PT_access'] = 'Walk2PT'

In [ ]:
##PNR KNR W2PT seven
empty = mode_filt.MODELMODE == 0
mode_filt.loc[mode_seven & empty, 'MODELMODE'] = mode_filt.loc[mode_seven & empty, 'MAINMODE']

walk2 = mode_filt.MODE1 == 'Walking'
drive2 = mode_filt.MODE1 == 'Car driver'
dropped2 = mode_filt.MODE1 == 'Car passenger'
mode_filt.loc[mode_seven & empty & p_to_a & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_seven & empty & p_to_a & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_seven & empty & p_to_a & dropped2, 'PT_access'] = 'KnR'
mode_filt.loc[mode_seven & empty & O_to_D & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_seven & empty & O_to_D & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_seven & empty & O_to_D & dropped2, 'PT_access'] = 'KnR'
walk3 = mode_filt.MODE7 == 'Walking'
drive3 = mode_filt.MODE7 == 'Car driver'
dropped3 = mode_filt.MODE7 == 'Car passenger'
mode_filt.loc[mode_seven & empty & a_to_p & walk3, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_seven & empty & a_to_p & drive3, 'PT_access'] = 'PnR'
mode_filt.loc[mode_seven & empty & a_to_p & dropped3, 'PT_access'] = 'KnR'

empty_pt_ac = mode_filt.PT_access == 'None'
mode_filt.loc[mode_seven & empty & empty_pt_ac, 'PT_access'] = 'Walk2PT'

In [ ]:
##PNR KNR W2PT eight
empty = mode_filt.MODELMODE == 0
mode_filt.loc[mode_eight & empty, 'MODELMODE'] = mode_filt.loc[mode_eight & empty, 'MAINMODE']

walk2 = mode_filt.MODE1 == 'Walking'
drive2 = mode_filt.MODE1 == 'Car driver'
dropped2 = mode_filt.MODE1 == 'Car passenger'
mode_filt.loc[mode_eight & empty & p_to_a & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_eight & empty & p_to_a & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_eight & empty & p_to_a & dropped2, 'PT_access'] = 'KnR'
mode_filt.loc[mode_eight & empty & O_to_D & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_eight & empty & O_to_D & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_eight & empty & O_to_D & dropped2, 'PT_access'] = 'KnR'
walk3 = mode_filt.MODE8 == 'Walking'
drive3 = mode_filt.MODE8 == 'Car driver'
dropped3 = mode_filt.MODE8 == 'Car passenger'
mode_filt.loc[mode_eight & empty & a_to_p & walk3, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_eight & empty & a_to_p & drive3, 'PT_access'] = 'PnR'
mode_filt.loc[mode_eight & empty & a_to_p & dropped3, 'PT_access'] = 'KnR'

empty_pt_ac = mode_filt.PT_access == 'None'
mode_filt.loc[mode_eight & empty & empty_pt_ac, 'PT_access'] = 'Walk2PT'

# del empty_pt_ac, walk2, walk3, drive2, drive3, dropped2, dropped3, empty
# del a_to_p, p_to_a, O_to_D, mode_one, mode_two, mode_three, mode_four
# del mode_five, mode_six, mode_seven, mode_eight
# del car_driver, car_pass, motorcycle_driver, motorcycle_pass
# del bicycle
# del not_car, not_car_p, ride_share, single_mode, taxi
# del train, truck_driver, truck_pass, uber, walk

print('Number of Trip Records in Original: {}'.format(len(TRIPS.axes[0])))
print('Number of Trip Records: {}'.format(len(mode_filt.axes[0])))
trip = mode_filt
del mode_filt

In [ ]:
trip_non_work_flag = trip['Dependent'] == 1
trip_worker_flag = trip['Worker'] == 1
trip_student_flag = trip['Student'] == 1
trip_non_student_flag = trip['Student'] != 1
trip_worker_b_flag = trip['Worker_Blue'] == 1
trip_worker_w_flag = trip['Worker_White'] == 1
trip_student_t_flag = trip['Student_Tert'] == 1
trip_student_s_flag = trip['Student_Sec'] == 1
trip_student_p_flag = trip['Student_Prim'] == 1

In [ ]:
trip.loc[trip_non_work_flag & trip_non_student_flag, 'Non-work, non-study'] = 1
trip.loc[trip_non_work_flag & trip_non_student_flag, 'Demog Classification' ] = 'Non-work, non-study'
trip.loc[trip_non_work_flag & trip_non_student_flag, 'Demog Classification2' ] = 'Non-work, non-study'

trip.loc[trip_worker_flag & trip_non_student_flag, 'Work, non-study'] = 1
trip.loc[trip_worker_flag & trip_non_student_flag, 'Demog Classification' ] = 'Work, non-study'
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_non_student_flag, 'Work_white, non-study'] = 1
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_non_student_flag, 'Demog Classification2' ] = 'Work_white, non-study'
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_non_student_flag, 'Work_blue, non-study'] = 1
trip.loc[trip_worker_flag & trip_worker_b_flag & trip_non_student_flag, 'Demog Classification2' ] = 'Work_blue, non-study'

trip.loc[trip_worker_flag & trip_student_flag, 'Work, study'] = 1
trip.loc[trip_worker_flag & trip_student_flag, 'Demog Classification' ] = 'Work, study'
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_student_flag & trip_student_t_flag, 'Work_White, study_tert'] = 1
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_student_flag & trip_student_t_flag, 'Demog Classification2' ] = 'Work_white, study_tert'
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_student_flag & trip_student_t_flag, 'Work_Blue, study_tert'] = 1
trip.loc[trip_worker_flag & trip_worker_b_flag & trip_student_flag & trip_student_t_flag, 'Demog Classification2' ] = 'Work_Blue, study_tert'
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_student_flag & trip_student_s_flag, 'Work_White, study_sec'] = 1
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_student_flag & trip_student_s_flag, 'Demog Classification2' ] = 'Work_white, study_sec'
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_student_flag & trip_student_s_flag, 'Work_Blue, study_sec'] = 1
trip.loc[trip_worker_flag & trip_worker_b_flag & trip_student_flag & trip_student_s_flag, 'Demog Classification2' ] = 'Work_Blue, study_sec'

trip.loc[trip_non_work_flag & trip_student_flag, 'Non-work, study'] = 1
trip.loc[trip_non_work_flag & trip_student_flag, 'Demog Classification' ] = 'Non-work, study'
trip.loc[trip_non_work_flag & trip_student_flag & trip_student_p_flag, 'Non-work, study_prim'] = 1
trip.loc[trip_non_work_flag & trip_student_flag & trip_student_p_flag, 'Demog Classification2' ] = 'Non-work, study_prim'
trip.loc[trip_non_work_flag & trip_student_flag & trip_student_p_flag, 'Non-work, study_sec'] = 1
trip.loc[trip_non_work_flag & trip_student_flag & trip_student_p_flag, 'Demog Classification2' ] = 'Non-work, study_sec'
trip.loc[trip_non_work_flag & trip_student_flag & trip_student_p_flag, 'Non-work, study_tert'] = 1
trip.loc[trip_non_work_flag & trip_student_flag & trip_student_p_flag, 'Demog Classification2' ] = 'Non-work, study_tert'

In [ ]:
%%time
"""Export Tables"""
pivot_ui(trip, vals=['TRIPWGT19'], aggregatorName='Sum', outfile_path='Trips_pivot.html')
pivot_ui(Household, vals=['PERSWGT19'], aggregatorName='Sum', outfile_path='Households_pivot.html')
pivot_ui(Person, vals=['PERSWGT19'], aggregatorName='Sum', outfile_path='Persons_pivot.html')
trip.to_csv('Trips_table.csv')
Household.to_csv('Households_table.csv')
Person.to_csv('Persons_table.csv')
